## Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

### Exercise: Fine-Tuning SmolLM2 with SFTTrainer

Take a dataset from the Hugging Face hub and finetune a model on it.

Difficulty Levels

🐢 Use the `HuggingFaceTB/smoltalk` dataset

🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.

🦁 Select a dataset that relates to a real world use case your interested in

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

import json
from trl import setup_chat_format, SFTConfig, SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer

True
Tesla P40


In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

In [ ]:
# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

### Generate with the base model

Here we will try out the base model which does not have a chat template.

In [ ]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role":"user","content":prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
print(formatted_prompt)

<|im_start|>user
Write a haiku about programming<|im_end|>



In [ ]:
# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
print(inputs)

{'input_ids': tensor([[    1,  4093,   198, 19161,   253,   421, 30614,   563,  6256,     2,
           198]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [ ]:
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


### Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates**. They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [ ]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

In [ ]:
ds['train'][0]['messages']

[{'content': 'Hi there', 'role': 'user'},
 {'content': 'Hello! How can I help you today?', 'role': 'assistant'},
 {'content': "I'm looking for a beach resort for my next vacation. Can you recommend some popular ones?",
  'role': 'user'},
 {'content': "Some popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.",
  'role': 'assistant'},
 {'content': 'That sounds great. Are there any resorts in the Caribbean that are good for families?',
  'role': 'user'},
 {'content': 'Yes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.',
  'role': 'assistant'},
 {'content': "Okay, I'll look into those. Thanks for the recommendations!",
  'role': 'user'},
 {'content': "You're welcome. I hope you find the perfect resort for your vacation.",
  'role': 'assistant'}]

In [ ]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template,
# you will need to process it. Refer to the [module](../chat_templates.md)

def process_dataset(sample):
    # Convert the sample into a chat format and tokenize it
    formatted_prompts = tokenizer.apply_chat_template(sample['messages'], tokenize=False)
    tokenized_output = tokenizer(formatted_prompts)

    # Remove unused columns (only keeping tokenized fields)
    keys_to_keep = ["input_ids", "attention_mask"]  # Adjust based on tokenizer output
    tokenized_output = {key: tokenized_output[key] for key in keys_to_keep if key in tokenized_output}

    return tokenized_output

ds = ds.map(process_dataset, remove_columns=ds['train'].column_names)

ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 119
    })
})

### Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [ ]:


# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=100,  # Frequency of evaluation
    fp16=True,
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model,
    report_to = "none"
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

/home/loc/miniconda3/envs/py3x/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_3978/3721637931.py:20: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


### Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [ ]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

  0%|          | 0/1000 [00:00<?, ?it/s]

{'loss': 1.7802, 'grad_norm': 2.812790870666504, 'learning_rate': 4.9550000000000005e-05, 'epoch': 0.02}
{'loss': 1.2665, 'grad_norm': 2.6464433670043945, 'learning_rate': 4.905e-05, 'epoch': 0.04}
{'loss': 1.2406, 'grad_norm': 2.409515380859375, 'learning_rate': 4.855e-05, 'epoch': 0.05}
{'loss': 1.166, 'grad_norm': 2.539659023284912, 'learning_rate': 4.805e-05, 'epoch': 0.07}
{'loss': 1.0683, 'grad_norm': 2.2946698665618896, 'learning_rate': 4.755e-05, 'epoch': 0.09}
{'loss': 1.1578, 'grad_norm': 2.285670042037964, 'learning_rate': 4.705e-05, 'epoch': 0.11}
{'loss': 1.1655, 'grad_norm': 2.017538070678711, 'learning_rate': 4.655000000000001e-05, 'epoch': 0.12}
{'loss': 1.1257, 'grad_norm': 2.1274728775024414, 'learning_rate': 4.605e-05, 'epoch': 0.14}
{'loss': 1.0882, 'grad_norm': 2.2429051399230957, 'learning_rate': 4.555e-05, 'epoch': 0.16}
{'loss': 1.1118, 'grad_norm': 2.083092451095581, 'learning_rate': 4.5050000000000004e-05, 'epoch': 0.18}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.1242440938949585, 'eval_runtime': 4.8793, 'eval_samples_per_second': 24.389, 'eval_steps_per_second': 3.074, 'epoch': 0.18}
{'loss': 1.0627, 'grad_norm': 2.0681509971618652, 'learning_rate': 4.4550000000000005e-05, 'epoch': 0.19}
{'loss': 1.124, 'grad_norm': 2.0806641578674316, 'learning_rate': 4.405e-05, 'epoch': 0.21}
{'loss': 1.0924, 'grad_norm': 2.004514694213867, 'learning_rate': 4.355e-05, 'epoch': 0.23}
{'loss': 1.064, 'grad_norm': 2.1443967819213867, 'learning_rate': 4.305e-05, 'epoch': 0.25}
{'loss': 1.0628, 'grad_norm': 2.0638463497161865, 'learning_rate': 4.2550000000000004e-05, 'epoch': 0.27}
{'loss': 1.0683, 'grad_norm': 2.015770673751831, 'learning_rate': 4.205e-05, 'epoch': 0.28}
{'loss': 1.0975, 'grad_norm': 2.063107490539551, 'learning_rate': 4.155e-05, 'epoch': 0.3}
{'loss': 1.0372, 'grad_norm': 1.7578182220458984, 'learning_rate': 4.105e-05, 'epoch': 0.32}
{'loss': 1.0558, 'grad_norm': 1.998705267906189, 'learning_rate': 4.055e-05, 'epoch': 0.34}
{'lo

  0%|          | 0/15 [00:00<?, ?it/s]

...

{'eval_loss': 1.0248416662216187, 'eval_runtime': 4.8223, 'eval_samples_per_second': 24.677, 'eval_steps_per_second': 3.111, 'epoch': 1.42}
{'loss': 0.7991, 'grad_norm': 1.8500282764434814, 'learning_rate': 9.55e-06, 'epoch': 1.43}
{'loss': 0.7896, 'grad_norm': 1.859580636024475, 'learning_rate': 9.05e-06, 'epoch': 1.45}
{'loss': 0.8112, 'grad_norm': 1.7300654649734497, 'learning_rate': 8.550000000000001e-06, 'epoch': 1.47}
{'loss': 0.8285, 'grad_norm': 1.7986068725585938, 'learning_rate': 8.050000000000001e-06, 'epoch': 1.49}
{'loss': 0.777, 'grad_norm': 1.7774221897125244, 'learning_rate': 7.55e-06, 'epoch': 1.5}
{'loss': 0.7905, 'grad_norm': 1.872243046760559, 'learning_rate': 7.049999999999999e-06, 'epoch': 1.52}
{'loss': 0.7849, 'grad_norm': 1.9118177890777588, 'learning_rate': 6.550000000000001e-06, 'epoch': 1.54}
{'loss': 0.8348, 'grad_norm': 1.8157360553741455, 'learning_rate': 6.0500000000000005e-06, 'epoch': 1.56}
{'loss': 0.8138, 'grad_norm': 1.6830424070358276, 'learning_ra

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.022844910621643, 'eval_runtime': 4.8164, 'eval_samples_per_second': 24.707, 'eval_steps_per_second': 3.114, 'epoch': 1.59}
{'loss': 0.8333, 'grad_norm': 1.7024340629577637, 'learning_rate': 4.5500000000000005e-06, 'epoch': 1.61}
{'loss': 0.8066, 'grad_norm': 1.8077661991119385, 'learning_rate': 4.05e-06, 'epoch': 1.63}
{'loss': 0.8103, 'grad_norm': 2.0675742626190186, 'learning_rate': 3.55e-06, 'epoch': 1.65}
{'loss': 0.7929, 'grad_norm': 1.7160780429840088, 'learning_rate': 3.05e-06, 'epoch': 1.66}
{'loss': 0.8618, 'grad_norm': 1.9925400018692017, 'learning_rate': 2.55e-06, 'epoch': 1.68}
{'loss': 0.8606, 'grad_norm': 1.822501301765442, 'learning_rate': 2.0500000000000003e-06, 'epoch': 1.7}
{'loss': 0.78, 'grad_norm': 1.732309341430664, 'learning_rate': 1.55e-06, 'epoch': 1.72}
{'loss': 0.7955, 'grad_norm': 1.7780712842941284, 'learning_rate': 1.0500000000000001e-06, 'epoch': 1.73}
{'loss': 0.8346, 'grad_norm': 1.7961950302124023, 'learning_rate': 5.5e-07, 'epoch': 1.7

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0213946104049683, 'eval_runtime': 4.8115, 'eval_samples_per_second': 24.732, 'eval_steps_per_second': 3.118, 'epoch': 1.77}
{'train_runtime': 705.8145, 'train_samples_per_second': 5.667, 'train_steps_per_second': 1.417, 'train_loss': 0.9620923929214478, 'epoch': 1.77}


In [ ]:
# trainer.push_to_hub(tags=finetune_tags)

### Bonus Exercise: Generate with fine-tuned model

🐕 Use the fine-tuned to model generate a response, just like with the base example..


In [12]:
# Test the fine-tuned model on the same prompt
finetune_name = "SmolLM2-FT-MyDataset"
# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=finetune_name
).to(device)

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=finetune_name)

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.
outputs = model.generate(**inputs)

print(outputs)

# Decode and print output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

tensor([[    1,  4093,   198, 19161,   253,   421, 30614,   563,  6256,     2,
           198,     1,   520,  9531,   198, 19556,    17,  1073,   416,   339,
           724,   346,  1834,    47,   339,  5248,   253,  1789,  1743,   284,
           339]], device='cuda:0')
user
Write a haiku about programming
assistant
Hello! How can I help you today? I'm a language model and I



💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the HuggingFaceTB/SmolLM2-135M model using the SFTTrainer. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.